# <font color=blue>Animal Attacks in North America
***
***

In [54]:
#import packages
from lxml import html
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import re

In [55]:
#function to get rows in a table
def get_table_rows(table):
    l = []
    table_rows = table.find_all('tr')
    for tr in table_rows:
        td = tr.find_all('td')
        row = [tr.text for tr in td]
        l.append(row)
    return l

In [56]:
#function to get rows in table after marker
def get_rows_table(tables):
    tb=[]
    for barbie in tables:
        tables=barbie.findNext('table')
        tb.append(tables)
    l = []

    for table in tb:
        if table is not None:
            table_rows = table.find_all('tr')
            row = []
            for tr in table_rows:
                td = tr.find_all('td')
                row = [tr.text for tr in td]
                l.append(row)
    return l

In [57]:
def get_rows_all_tables(tables):
    l=[]
    for tb in tables:
        table_rows = tb.find_all('tr')
        for tr in table_rows:
            td = tr.find_all('td')
            row = [tr.text for tr in td]
            l.append(row)
    return l

In [58]:
# get all locations hrefs in the n-th column
def extract_href_location(tables,n):
    l=[]
    for tb in tables:
        table_rows = tb.find_all('tr')
        for tr in table_rows:
            td = tr.find_all('td')
            row = [tr.text for tr in td]
            for tt in td:
                ref=tt.find('a', href=re.compile(r'.*wiki*'))
                if ref:
                    title = ref.get('title')
                    l.append(title)
                else:
                    l.append(np.nan)
    return l[n-1::n]

In [59]:
#install packages
#!pip install geocoder
#!pip install folium==0.5.0
#!conda install -c conda-forge folium=0.5.0 --yes
#!pip install geopy

In [60]:
#import packages to draw on map
import geocoder
import folium
from geopy.geocoders import Nominatim

In [61]:
# create map of North America using latitude and longitude values
address = 'Vancouver, BC'

geolocator = Nominatim(user_agent="na_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
map_north_america = folium.Map(location=[latitude, longitude], zoom_start=3)
map_north_america_pure = map_north_america

In [62]:
def add_coordinates(table):
    lat_lng_coords = None
    latlong=[]
    for n in range(0,len(table)):
        g = geocoder.arcgis(table['Location'].values[n])
        lat_lng_coords = g.latlng
        latlong.append(lat_lng_coords)
    laa = np.array(latlong)
    latitude=laa[:,0].tolist()
    longitude=laa[:,1].tolist()
    lat=pd.DataFrame(latitude,columns=['Latitude']) 
    long=pd.DataFrame(longitude,columns=['Longitude']) 
    lulo=pd.concat([lat,long], axis=1)
    data=pd.concat([table,lulo],axis=1)
    return data

In [63]:
def add_to_map(table,colour,fill_colour,map_of):
    for lat, lng, Loc in zip(table['Latitude'], table['Longitude'], table['Location']):
        label = '{}'.format(Loc)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker([lat, lng], radius=3, popup=label, color=colour, fill=True, fill_color=fill_colour, fill_opacity=0.7, parse_html=False).add_to(map_of)
    return map_of

In [64]:
def add_categorical_legend(folium_map, title, colors, labels):
    if len(colors) != len(labels):
        raise ValueError("colors and labels must have the same length.")

    color_by_label = dict(zip(labels, colors))
    
    legend_categories = ""     
    for label, color in color_by_label.items():
        legend_categories += f"<li><span style='background:{color}'></span>{label}</li>"
        
    legend_html = f"""
    <div id='maplegend' class='maplegend'>
      <div class='legend-title'>{title}</div>
      <div class='legend-scale'>
        <ul class='legend-labels'>
        {legend_categories}
        </ul>
      </div>
    </div>
    """
    script = f"""
        <script type="text/javascript">
        var oneTimeExecution = (function() {{
                    var executed = false;
                    return function() {{
                        if (!executed) {{
                             var checkExist = setInterval(function() {{
                                       if ((document.getElementsByClassName('leaflet-top leaflet-right').length) || (!executed)) {{
                                          document.getElementsByClassName('leaflet-top leaflet-right')[0].style.display = "flex"
                                          document.getElementsByClassName('leaflet-top leaflet-right')[0].style.flexDirection = "column"
                                          document.getElementsByClassName('leaflet-top leaflet-right')[0].innerHTML += `{legend_html}`;
                                          clearInterval(checkExist);
                                          executed = true;
                                       }}
                                    }}, 100);
                        }}
                    }};
                }})();
        oneTimeExecution()
        </script>
      """
   

    css = """

    <style type='text/css'>
      .maplegend {
        z-index:9999;
        float:right;
        background-color: rgba(255, 255, 255, 1);
        border-radius: 5px;
        border: 2px solid #bbb;
        padding: 10px;
        font-size:12px;
        positon: relative;
      }
      .maplegend .legend-title {
        text-align: left;
        margin-bottom: 5px;
        font-weight: bold;
        font-size: 90%;
        }
      .maplegend .legend-scale ul {
        margin: 0;
        margin-bottom: 5px;
        padding: 0;
        float: left;
        list-style: none;
        }
      .maplegend .legend-scale ul li {
        font-size: 80%;
        list-style: none;
        margin-left: 0;
        line-height: 18px;
        margin-bottom: 2px;
        }
      .maplegend ul.legend-labels li span {
        display: block;
        float: left;
        height: 16px;
        width: 30px;
        margin-right: 5px;
        margin-left: 0;
        border: 0px solid #ccc;
        }
      .maplegend .legend-source {
        font-size: 80%;
        color: #777;
        clear: both;
        }
      .maplegend a {
        color: #777;
        }
    </style>
    """

    folium_map.get_root().header.add_child(folium.Element(script + css))

    return folium_map

#  Bear Attacks
***

Scrape and Explore Dataset

In [86]:
r = requests.get("https://en.wikipedia.org/wiki/List_of_fatal_bear_attacks_in_North_America")
soup = BeautifulSoup(r.content, "html.parser")

In [87]:
dflist=[]
for bear_type in ("Polar","Black","Brown"):
    bears=soup.findAll(title="Edit section: "+bear_type+" bear")
    bear=get_rows_table(bears)
    df=pd.DataFrame(bear, columns=["Person", "Date", "Type", "Location", "Details"])
    df['Bear']=bear_type
    dflist.append(df)
all_attacks=pd.concat(dflist).replace(r'\n',' ', regex=True).dropna().reset_index(drop=True)
all_attacks.head()

,Person,Date,Type,Location,Details,Bear
0,"Darryl Kaunak, 33, male","August 23, 2018",Wild,"Lyon Inlet, Nunavut","Three men from Naujaat, whose boat had broken ...",Polar
1,"Aaron Gibbons, 31, male","July 3, 2018",Wild,"Sentry Island, Nunavut",A polar bear approached a man and his children...,Polar
2,"Hattie Amitnak, 64, female","July 9, 1999",Wild,"near Rankin Inlet, Nunavut",Amitnak was mauled after trying to distract a ...,Polar
3,"Carl Stalker, 28, male","December 8, 1990",Wild,"Point Lay, Alaska","While Stalker was walking with his girlfriend,...",Polar
4,"Juan Perez, 11, male","May 19, 1987",Captive,"Brooklyn, New York",Perez was killed by two bears after climbing a...,Polar


In [67]:
data=add_coordinates(all_attacks)
data=data.loc[data['Type']=='Wild'].reset_index(drop=True)
pb=data.loc[data['Bear']=='Polar']
blb=data.loc[data['Bear']=='Black']
brb=data.loc[data['Bear']=='Brown']

In [68]:
map_bear_attacks=add_to_map(brb,'brown','#3186cc',add_to_map(blb,'black','#3186cc',add_to_map(pb,'blue','#3186cc',map_north_america)))
map_bear_attacks;

# Wolf Attacks
***

In [88]:
r = requests.get("https://en.wikipedia.org/wiki/List_of_wolf_attacks_in_North_America")
soup = BeautifulSoup(r.content, "html.parser")

In [89]:
fatals=soup.find('table')
df=pd.DataFrame(get_rows_table(fatals), columns=["Victim(s)","Age","Sex","Date","Type of attack","Location","Details","Source(s)"])
ftl_wolf_attacks = df.replace(r'\n',' ', regex=True).dropna().reset_index(drop=True)
ftl_wolf_attacks.head()

,Victim(s),Age,Sex,Date,Type of attack,Location,Details,Source(s)
0,Candice Berner,32,♀,8 March 2010,Predatory,"Chignik, Alaska, US, 75 miles southwest of Kodiak","Berner, a teacher and avid jogger, was discove...","Findings, Alaska Department of Fish and Game[1]"
1,Kenton Carnegie,22,♂,8 November 2005,Predatory,"Points North Landing, Saskatchewan, Canada","In the weeks leading up to the assault, natura...","Dr. Valerius Geist, University of Calgary;[2][..."
2,Patricia Wyman,23–24,♀,18 April 1996,Captive,"Haliburton Forest, Haliburton County, Ontario,...",Wyman was a wildlife biologist who worked as a...,Dr. Erich Klinghammer[5]
3,Alyshia Berczyk,3,♀,3 June 1989,Captive,"Forest Lake, Minnesota, US",By her family's wolf in the backyard of her fa...,"Rochester, Minnesota Post-Bulletin[6]"
4,Inuit boy,Child,♂,1943,Rabid,"Wainwright, Alaska",Died of rabies from a wolf bite.,"NINA: Norsk institutt for naturforskning ""The ..."


In [71]:
wvs=add_coordinates(ftl_wolf_attacks)

In [72]:
#add_to_map(wvs,'gray','gray',map_north_america)
add_to_map(wvs,'gray','gray',map_north_america_pure);

# Cougar Attacks
***

In [90]:
r = requests.get("https://en.wikipedia.org/wiki/List_of_fatal_cougar_attacks_in_North_America")
soup = BeautifulSoup(r.content, "html.parser")

In [91]:
cougars=soup.findAll('table',{"class":"wikitable sortable"})
puma=[x for x in get_rows_all_tables(cougars) if x != []]
cgr_attacks=pd.DataFrame(puma, columns=["Name, Age, Gender", "Date","Location"]).replace(r'\n',' ', regex=True).dropna().reset_index(drop=True)
cgr_attacks.head()

,"Name, Age, Gender",Date,Location
0,"(Unnamed Child), 3","August 24, 1868","""Killed by a Cougar.— The Oregon Harold of Aug..."
1,"Arthur Dangle, 7","June 19, 1890",Killed and eaten by two cougars while playing ...
2,"Frank Cook, age unknown","November 11, 1901",Attacked by a cougar east of the Santa Caterin...
3,"A. C. Marklein, age unknown","March 1, 1904",Killed by a cougar in Bushy Cane Creek Magoffi...
4,"Child Brown, 2 or 14 *NOTE: Probable false rep...","January 31, 1909","Killed by a cougar near Balboa, California. Th..."


In [75]:
# Extract Age
# cgr_attacks['Name, Age, Gender'].str.extract(r'([\d]+)',expand=False)

In [76]:
# Overwrite Location with href-Location-data
cgr_attacks["Location"]=pd.DataFrame(extract_href_location(cougars,3))

In [77]:
cuga=add_coordinates(cgr_attacks)

In [78]:
add_to_map(cuga,'yellow','gray',map_north_america);

# Snake bites in the US
***

In [79]:
r = requests.get("https://en.wikipedia.org/wiki/List_of_fatal_snake_bites_in_the_United_States")
soup = BeautifulSoup(r.content, "html.parser")

In [80]:
snakes=soup.findAll('table')
snk=[x for x in get_rows_all_tables(snakes) if x != []]
snk_bites=pd.DataFrame(snk, columns=["Name, Age, Gender", "Date","Species","Location"]).replace(r'\n',' ', regex=True).dropna().reset_index(drop=True)

In [81]:
snk_bites["Location"]=pd.DataFrame(extract_href_location(snakes,4)).replace('Snake handling',np.nan).replace('Sistrurus catenatus',np.nan)
#snk_bites['Location'][6].partition("in ")[2].partition(".")[0]
snk_bites

,"Name, Age, Gender",Date,Species,Location
0,"Priscilla Meridith, 62, female","June 12, 2019",Timber rattlesnake,"Waverly, Georgia"
1,"Oliver ""Chum"" Baker, 52, male","May 25, 2019",Copperhead,"Winston County, Alabama"
2,"Lawrence Walters, 70, male","June 4, 2018",Rattlesnake,"Lawrence County, South Dakota"
3,"Barry Lester, 57, male","April 29, 2018",Rattlesnake,"Osage County, Oklahoma"
4,"Daniel Hohs, 31, male","October 7, 2017",Rattlesnake,"Golden, Colorado"
...,...,...,...,...
102,"Maggie Lee, female","October 24, 1854",Rattlesnake,NaN
103,"H. M. Pettigrew, 31, male","August 15, 1841",Rattlesnake,"Fannin County, Texas"
104,"Richardson, infant son of Wm. & Ella",1796,Rattlesnake,NaN
105,Unknown person,1791,Timber rattlesnake,Massachusetts


In [82]:
snkb=snk_bites.dropna().reset_index(drop=True)
snky=add_coordinates(snkb)

In [83]:
aaa=add_to_map(snky,'pink','pink',map_north_america)

In [84]:
map_all_attacks=add_categorical_legend(aaa, 'Animal Attacks in Northamerica',
                             colors = ['brown','black','blue','gray','yellow','pink'],
                           labels = ['Brown Bear', 'Black Bear','Polar Bear','Wolf','Cougar','Snake'])
map_all_attacks